# Data Acquisition

This notebook downloads the data from the _OpenLigaDB_ API and prepares pandas dataframes.
The data consists of two types:
- matches
- tables

### Download matches

In [1]:
import requests
import json
from tqdm.notebook import trange, tqdm

In [2]:
SEASON_START = 2010
SEASON_END = 2024

matches = []
for season in trange(SEASON_START, SEASON_END + 1):
    response = requests.get(f"https://api.openligadb.de/getmatchdata/bl1/{season}")
    matches += response.json()

with open("data/matches.json", "w") as file:
    file.write(json.dumps(matches))

print(f"Downloaded {len(matches)} matches")
del matches

  0%|          | 0/15 [00:00<?, ?it/s]

Downloaded 4590 matches


### Download tables

In [3]:
tables = {}
for season in trange(SEASON_START, SEASON_END + 1):
    response = requests.get(f"https://api.openligadb.de/getbltable/bl1/{season}")
    tables[str(season)] = response.json()


with open("data/tables.json", "w") as file:
    file.write(json.dumps(tables))

del tables

  0%|          | 0/15 [00:00<?, ?it/s]

## Prepare Dataset

In [4]:
import pandas as pd

with open("data/matches.json", "r") as file:
    matches = json.load(file)


def retrieve_end_result(row: dict):
    results = row["matchResults"]
    end_result = next(
        (result for result in results if result.get("resultName") == "Endergebnis"),
        None,
    )

    if end_result is None:
        raise ValueError(f"Couldn't retrieve end result for {row}")

    return end_result


def match_to_record(row):
    record = {
        "id": row["matchID"],
        "match_day": row["group"]["groupOrderID"],
        "season": row["leagueSeason"],
        "host_id": row["team1"]["teamId"],
        "host_name": row["team1"]["shortName"] or row["team1"]["teamName"],
        "guest_id": row["team2"]["teamId"],
        "guest_name": row["team2"]["shortName"] or row["team2"]["teamName"],
    }
    if row["leagueSeason"] != 2024:
        result = retrieve_end_result(row)
        record["host_goals"] = result["pointsTeam1"]
        record["guest_goals"] = result["pointsTeam2"]
    else:
        try:
            result = retrieve_end_result(row)
            record["host_goals"] = result["pointsTeam1"]
            record["guest_goals"] = result["pointsTeam2"]
        except ValueError:
            record["host_goals"] = None
            record["guest_goals"] = None

    return record


records = [match_to_record(match) for match in matches]
del matches
df_matches_all = pd.DataFrame.from_records(records)
df_matches_all = df_matches_all.set_index("id")
df_matches_all.tail(2)

,match_day,season,host_id,host_name,guest_id,guest_name,host_goals,guest_goals
id,,,,,,,,
72518,34,2024,87,Gladbach,131,Wolfsburg,NaN,NaN
72519,34,2024,98,St. Pauli,129,Bochum,NaN,NaN


In [5]:
# Replace long team name with shorter one
df_matches_all.loc[
    df_matches_all["host_name"] == "TSG 1899 Hoffenheim", "host_name"
] = "Hoffenheim"
df_matches_all.loc[
    df_matches_all["guest_name"] == "TSG 1899 Hoffenheim", "guest_name"
] = "Hoffenheim"

In [6]:
with open("data/tables.json", "r") as file:
    tables = json.load(file)

columns = ['points', 'opponentGoals', 'goals', 'matches', 'won', 'lost', 'draw', 'goalDiff']

team_rows = [
    [col, year, team['teamInfoId'], team[col]]
    for year, teams in tables.items()
    for team in teams
    for col in columns
]

df_teams_all = pd.DataFrame(team_rows, columns=['feature', 'year', 'team', 'value'])
del team_rows
df_teams_all = df_teams_all.groupby(["team", "year", "feature"]).sum()["value"].reset_index()
df_teams_all = pd.pivot_table(
    df_teams_all, values="value", index=["team", "year"], columns="feature"
)
df_teams_all = df_teams_all.rename(
    columns={
        "draw": "draws",
        "won": "wins",
        "lost": "defeats",
        "goalDiff": "goal_diff",
        "opponentGoals": "opponent_goals",
    }
)
df_teams_all = df_teams_all.reset_index()
df_teams_all = df_teams_all.astype(int)
df_teams_all.tail(3)

feature,team,year,draws,goal_diff,goals,defeats,matches,opponent_goals,points,wins
267,1635,2022,6,23,64,8,34,41,66,20
268,1635,2023,8,38,77,7,34,39,65,19
269,1635,2024,1,2,4,0,3,2,7,2


### Create horizon

In [7]:
HORIZON = 3

historic_columns = ["points", "goals", "opponent_goals", "goal_diff", "wins", "draws", "defeats"]
test_years = range(df_teams_all["year"].min(), df_teams_all["year"].max() + 1)

historic_mean = []
for team in df_teams_all.index.unique():
    team_data = df_teams_all[df_teams_all["team"] == team]
    years = team_data["year"].unique()

    for test_year in test_years:
        horizont_years = [y for y in range(test_year - HORIZON, test_year) if y in years]
        if not horizont_years:
            continue

        mask_horizon = df_teams_all["year"].isin(horizont_years)
        masked_data = team_data.loc[mask_horizon, historic_columns]
        mean_values = masked_data.mean()

        record = {"team": team, "year": test_year, "horizon_years": horizont_years}
        for col in historic_columns:
            record[f"horizon_{col}"] = mean_values[col]

        historic_mean.append(record)


df_teams_historic = pd.DataFrame.from_records(historic_mean)
df_teams_historic.head(5)

,team,year,horizon_years,horizon_points,horizon_goals,horizon_opponent_goals,horizon_goal_diff,horizon_wins,horizon_draws,horizon_defeats
0,6,2011,[2010],68.000000,64.000000,44.000000,20.000000,20.000000,8.000000,6.000000
1,6,2012,"[2010, 2011]",61.000000,58.000000,44.000000,14.000000,17.500000,8.500000,8.000000
2,6,2013,"[2010, 2011, 2012]",62.333333,60.333333,42.333333,18.000000,18.000000,8.333333,7.666667
3,6,2014,"[2011, 2012, 2013]",60.000000,59.000000,41.333333,17.666667,17.666667,7.000000,9.333333
4,6,2015,"[2012, 2013, 2014]",62.333333,62.333333,39.000000,23.333333,18.333333,7.333333,8.333333


### Enrich matches with both teams' horizons

In [8]:
df_team_host = df_teams_all.reset_index()
df_team_host["year"] += 1
df_team_host.columns = "host_last_season_" + df_team_host.columns
df_full = pd.merge(
    left=df_matches_all.reset_index(),
    left_on=["host_id", "season"],
    right=df_team_host,
    right_on=["host_last_season_team", "host_last_season_year"],
    how="left",
    validate="m:1",
)

df_team_guest = df_teams_all.reset_index()
df_team_guest["year"] += 1
df_team_guest.columns = "guest_last_season_" + df_team_guest.columns
df_full = pd.merge(
    left=df_full,
    left_on=["guest_id", "season"],
    right=df_team_guest,
    right_on=["guest_last_season_team", "guest_last_season_year"],
    how="left",
    validate="m:1",
)

df_full = df_full.drop(
    columns=[
        "guest_last_season_team",
        "host_last_season_team",
        "guest_last_season_year",
        "host_last_season_year",
    ]
)

df_team_host = df_teams_historic.copy()
df_team_host.columns = "host_" + df_team_host.columns
df_full = pd.merge(
    left=df_full,
    left_on=["host_id", "season"],
    right=df_team_host,
    right_on=["host_team", "host_year"],
    how="left",
    validate="m:1",
)

df_team_guest = df_teams_historic.copy()
df_team_guest.columns = "guest_" + df_team_guest.columns
df_full = pd.merge(
    left=df_full,
    left_on=["guest_id", "season"],
    right=df_team_guest,
    right_on=["guest_team", "guest_year"],
    how="left",
    validate="m:1",
)

df_full = df_full.drop(
    columns=[
        "guest_team",
        "guest_year",
        "host_team",
        "host_year",
        "guest_horizon_years",
        "host_horizon_years",
        "host_last_season_index",
        "guest_last_season_index",
    ]
)
df_full = df_full.set_index("id")
df_full.tail(5)

,match_day,season,host_id,host_name,guest_id,guest_name,host_goals,guest_goals,host_last_season_draws,host_last_season_goal_diff,...,host_horizon_wins,host_horizon_draws,host_horizon_defeats,guest_horizon_points,guest_horizon_goals,guest_horizon_opponent_goals,guest_horizon_goal_diff,guest_horizon_wins,guest_horizon_draws,guest_horizon_defeats
id,,,,,,,,,,,,,,,,,,,,,
72515,34,2024,112,Freiburg,91,Frankfurt,NaN,NaN,9.0,-13.0,...,14.333333,9.000000,10.666667,46.333333,51.333333,50.333333,1.000000,11.333333,12.333333,10.333333
72516,34,2024,95,Augsburg,80,Union Berlin,NaN,NaN,9.0,-10.0,...,9.666667,8.000000,16.333333,50.666667,44.666667,46.666667,-2.000000,14.333333,7.666667,12.000000
72517,34,2024,81,Mainz,6,Leverkusen,NaN,NaN,14.0,-12.0,...,10.666667,10.333333,13.000000,68.000000,75.333333,40.000000,35.333333,20.333333,7.000000,6.666667
72518,34,2024,87,Gladbach,131,Wolfsburg,NaN,NaN,13.0,-11.0,...,10.000000,10.666667,13.333333,42.666667,47.000000,52.666667,-5.666667,11.666667,7.666667,14.666667
72519,34,2024,98,St. Pauli,129,Bochum,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,36.666667,40.000000,66.000000,-26.000000,9.666667,7.666667,16.666667


## Imputation

In [9]:
cols_fill_zero = ["goals", "wins", "points", "matches"]
for col in df_full.columns:
    if not any([c for c in cols_fill_zero if c in col]):
        continue
    df_full[col] = df_full[col].fillna(0)

cols_fill_min = ["goal_diff", "draws"]
for col in df_full.columns:
    if not any([c for c in cols_fill_min if c in col]):
        continue
    df_full[col] = df_full[col].fillna(df_full[col].min())

cols_fill_max = ["defeats"]
for col in df_full.columns:
    if not any([c for c in cols_fill_max if c in col]):
        continue
    df_full[col] = df_full[col].fillna(df_full[col].max())

### Create _difference_ features

In [10]:
cols = [col.replace("host_", "") for col in df_full.columns if col.startswith("host")]
cols = [col for col in cols if col not in ["id", "name", "goals"]]

for col in cols:
    df_full["diff_" + col] = df_full["host_" + col] - df_full["guest_" + col]

## Splitting

In [11]:
df_train = df_full[df_full["season"] < 2023]
df_train.to_csv("data/matches_train.csv")

df_validation = df_full[df_full["season"] == 2023]
df_validation.to_csv("data/matches_validation.csv")

df_future = df_full[df_full["season"] == 2024]
df_future.to_csv("data/matches_future.csv")